# Workshop: Vložení datasetu do sandboxu Zenodo přes REST API
**Tento notebook demonstruje celý proces vytvoření záznamu a vložení dat do sandbox.zenodo.org**

In [ ]:
import requests
import json
import yaml
import time
from pathlib import Path

# Nastavení API
ZENODO_API_URL = "https://sandbox.zenodo.org/api"
ACCESS_TOKEN = "your_access_token_here"  # Nahraďte svým tokenem

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

print(f"🔧 Zenodo API: nastaveno!")

## Krok 1: Načtení metadat ze YAML souboru

In [ ]:
with open("sandbox-zenodo-metadata.yaml", 'r', encoding='utf-8') as f:
    metadata = yaml.safe_load(f)

# Opravíme datum (YAML ho parsuje jako date objekt)
if hasattr(metadata['publication_date'], 'strftime'):
    metadata['publication_date'] = metadata['publication_date'].strftime('%Y-%m-%d')

print("📄 Načtená metadata:")
metadata

## Krok 2: Vytvoření nového záznamu (deposition)

In [ ]:
response = requests.post(f"{ZENODO_API_URL}/deposit/depositions", 
                        headers=headers, json={})

deposition = response.json()
deposition_id = deposition['id']
bucket_url = deposition['links']['bucket']

print(f"✅ Záznam vytvořen: ID {deposition_id}")
print(f"Celá odpověď serveru:")
deposition

## Krok 3: Transformace YAML -> JSON

In [ ]:
def convert_to_zenodo(yaml_data):
    """Jednoduchá konverze do Zenodo formátu"""
    
    # Konverze data
    pub_date = yaml_data['publication_date']
    if hasattr(pub_date, 'strftime'):
        pub_date = pub_date.strftime('%Y-%m-%d')
    
    # Základní metadata (víme že fungují)
    zenodo_metadata = {
        'title': yaml_data['title'],
        'upload_type': yaml_data['upload_type'],
        'description': yaml_data['description'],
        'publication_date': pub_date,
        'language': yaml_data['language'],
        'creators': yaml_data['creators'],
        'keywords': yaml_data['keywords'],
        'license': yaml_data['license']
    }
    
    # Přidáme pouze pole která víme že fungují
    if 'subjects' in yaml_data:
        zenodo_metadata['subjects'] = yaml_data['subjects']
    
    if 'grants' in yaml_data:
        zenodo_metadata['grants'] = yaml_data['grants']
    
    if 'related_identifiers' in yaml_data:
        zenodo_metadata['related_identifiers'] = yaml_data['related_identifiers']
    
    if 'contributors' in yaml_data:
        zenodo_metadata['contributors'] = yaml_data['contributors']
    
    if 'communities' in yaml_data:
        zenodo_metadata['communities'] = yaml_data['communities']
    
    return zenodo_metadata

# Konvertujeme metadata
zenodo_metadata = convert_to_zenodo(metadata)

data = {'metadata': zenodo_metadata}

print("📤 Odesíláme JSON pro aktualizaci metadat:")
print(json.dumps(data, indent=2, ensure_ascii=False))

## Krok 4: Aktualizace metadat záznamu

In [ ]:
data = {'metadata': zenodo_metadata}

response = requests.put(f"{ZENODO_API_URL}/deposit/depositions/{deposition_id}",
                       headers=headers, json=data)

if response.status_code == 200:
    print(f"✅ Metadata aktualizována")
else:
    print(f"❌ Chyba při aktualizaci: {response.status_code}")
    print(f"Odpověď: {response.text}")

## Krok 5: Vytvoření ukázkových dat pro upload

In [ ]:
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# CSV soubor
csv_content = """id,name,value,category
1,Sample A,10.5,machine_learning
2,Sample B,15.2,image_classification
3,Sample C,8.7,shiba_inu"""

with open(data_dir / "dataset.csv", 'w') as f:
    f.write(csv_content)

# README
readme_content = """# Dataset pro workshop

Ukázkový dataset pro demonstraci Zenodo API.

## Soubory
- dataset.csv: Hlavní data
- README.md: Tento popis
"""

with open(data_dir / "README.md", 'w') as f:
    f.write(readme_content)

files_to_upload = [data_dir / "dataset.csv", data_dir / "README.md"]
print(f"📁 Vytvořeno {len(files_to_upload)} souborů")

## Krok 6: Upload souborů

In [ ]:
upload_headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}
uploaded_files = []

for file_path in files_to_upload:
    filename = file_path.name
    
    print(f"📤 Nahrávám: {filename}")
    
    with open(file_path, 'rb') as f:
        response = requests.put(f"{bucket_url}/{filename}", 
                               headers=upload_headers, data=f)
    
    if response.status_code in [200, 201]:
        file_info = response.json()
        uploaded_files.append(file_info)
        print(f"✅ {filename}: {file_info['size']} bytů")
    
    time.sleep(0.5)

print(f"\n📊 Nahráno celkem: {len(uploaded_files)} souborů")

## Krok 8: Publikování záznamu

In [ ]:
response = requests.post(f"{ZENODO_API_URL}/deposit/depositions/{deposition_id}/actions/publish",
                        headers=headers)

if response.status_code == 202:
    published_record = response.json()
    print("🎉 ZÁZNAM PUBLIKOVÁN!")
    print(f"DOI: {published_record['doi']}")
    print(f"URL: {published_record['links']['record_html']}")
else:
    print(f"❌ Chyba při publikování: {response.status_code}")

## Shrnutí

In [ ]:
print("📋 HOTOVO!")
print("=" * 30)
print(f"✅ Záznam: {deposition_id}")
print(f"✅ Soubory: {len(uploaded_files)}")
print(f"✅ DOI: {published_record.get('doi', 'N/A')}")
print(f"✅ Communities: Žádost odeslána")
print()
print("🔗 Zkontrolujte záznam na:")
print(f"https://sandbox.zenodo.org/records/{published_record.get('id', deposition_id)}")